In [1]:
print("hello")

hello


In [91]:
with open("defects.csv") as f:
    lines = f.readlines()
    defects = [line.strip().split(",") for line in lines[1:]]

# Defects never occur at precise integer coordinates, so a defect will never be shared by two tiles.
defect_counts = {}
for x, y in defects:
    x = int(float(x))
    if x not in defect_counts:
        defect_counts[x] = {"a": 0, "b": 0, "c": 0}

    defect_counts[x][y] += 1

print(len(defect_counts))

322


In [92]:
dough_length = 500

biscuits = {
    0: {"length": 4, "value": 6, "defects": {"a": 4, "b": 2, "c": 3}, "id": 0},
    1: {"length": 8, "value": 12, "defects": {"a": 5, "b": 4, "c": 4}, "id": 1},
    2: {"length": 2, "value": 1, "defects": {"a": 1, "b": 2, "c": 1}, "id": 2},
    3: {"length": 5, "value": 8, "defects": {"a": 2, "b": 3, "c": 2}, "id": 3},
}

In [65]:
import copy

def hill_climb(biscuits, defect_counts, dough_length, max_iterations=1000):
    current_solution = [{"total_value": 0, "path": []} for _ in range(dough_length + 1)]
    best_solution = copy.deepcopy(current_solution)

    for _ in range(max_iterations):
        for i in range(1, dough_length + 1):
            for biscuit in biscuits.values():
                length = biscuit["length"]
                if length <= i:
                    defect_counts_for_biscuit = {"a": 0, "b": 0, "c": 0}
                    for j in range(i - length, i):
                        if j in defect_counts:
                            for k in defect_counts[j]:
                                defect_counts_for_biscuit[k] += defect_counts[j][k]

                    if all(defect_counts_for_biscuit[k] <= biscuit["defects"][k] for k in defect_counts_for_biscuit):
                        if (
                            biscuit["value"] + current_solution[i - length]["total_value"] > current_solution[i]["total_value"]
                            and biscuit["value"] + current_solution[i - length]["total_value"] > current_solution[i - 1]["total_value"]
                        ):
                            current_solution[i] = {
                                "total_value": biscuit["value"] + current_solution[i - length]["total_value"],
                                "path": current_solution[i - length]["path"] + [(i - biscuit["length"], biscuit["id"])],
                            }
                        elif current_solution[i - 1]["total_value"] > current_solution[i]["total_value"] and current_solution[i - 1]["total_value"] > biscuit["value"] + current_solution[i - length]["total_value"]:
                            current_solution[i] = current_solution[i - 1]
                        else:
                            current_solution[i] = current_solution[i]

        if current_solution[-1]["total_value"] > best_solution[-1]["total_value"]:
            best_solution = copy.deepcopy(current_solution)

    return best_solution

# Example usage
best_solution = hill_climb(biscuits, defect_counts, dough_length)
print("Best Total Value:", best_solution[-1]["total_value"])
print("Best Solution:", best_solution[-1]["path"])


Best Total Value: 488
Best Solution: [(186, 0), (190, 3), (195, 3), (200, 3), (205, 3), (210, 3), (215, 3), (220, 3), (225, 0), (229, 0), (233, 0), (237, 3), (242, 0), (246, 0), (250, 3), (255, 3), (260, 3), (265, 3), (270, 3), (275, 0), (279, 0), (283, 3), (288, 3), (293, 1), (301, 1), (309, 3), (314, 0), (318, 3), (323, 3), (328, 1), (336, 3), (341, 0), (345, 0), (349, 3), (354, 1), (362, 0), (366, 0), (370, 0), (374, 0), (378, 3), (383, 3), (388, 3), (393, 3), (398, 0), (402, 3), (407, 3), (412, 3), (417, 3), (422, 3), (427, 0), (431, 0), (435, 0), (439, 3), (444, 0), (448, 3), (453, 0), (457, 3), (462, 3), (467, 3), (472, 0), (476, 0), (480, 0), (484, 0), (488, 0), (492, 0), (496, 0)]


In [83]:
def is_solution_valid(biscuits, defect_counts, dough_length, solution):
    for i, (position, biscuit_id) in enumerate(solution):
        if position < 0 or position + biscuits[biscuit_id]["length"] > dough_length:
            return False

        defect_counts_for_biscuit = {"a": 0, "b": 0, "c": 0}
        for j in range(position, position + biscuits[biscuit_id]["length"]):
            if j in defect_counts:
                for k in defect_counts[j]:
                    defect_counts_for_biscuit[k] += defect_counts[j][k]

        if i != 0 and position <= solution[i - 1][0] + biscuits[solution[i - 1][1]]["length"] - 1:
            return False

        for k in defect_counts_for_biscuit:
            if defect_counts_for_biscuit[k] > biscuits[biscuit_id]["defects"][k]:
                return False

    return True


In [67]:
is_valid = is_solution_valid(biscuits, defect_counts, dough_length, best_solution[-1]["path"])
print("Is the solution valid?", is_valid)


Is the solution valid? True


TypeError: is_valid_placement_optimized() missing 2 required positional arguments: 'biscuits' and 'dough_length'

In [81]:
def count_defects_in_range_efficient(start, end, defect_counts):
    """ Efficiently count the number of defects of each class in a specified range. """
    counts = {cls: 0 for cls in 'abc'}
    for cls in counts.keys():
        counts[cls] = sum(defect_counts[cls][start:end])
    return counts

def generate_neighbors_efficient(solution, defect_counts):
    """ Generate a limited set of neighboring solutions for efficiency. """
    neighbors = []
    for i in range(len(solution)):
        position, current_type = solution[i]
        for biscuit_type in biscuits:
            if biscuit_type != current_type:
                new_solution = solution[:i] + [(position, biscuit_type)] + solution[i+1:]
                valid = True
                for pos, bt in new_solution:
                    counts = count_defects_in_range_efficient(pos, pos + biscuits[bt]['length'], defect_counts)
                    if any(counts[cls] > biscuits[bt]['defects'][cls] for cls in counts):
                        valid = False
                        break
                if valid:
                    neighbors.append(new_solution)
    return neighbors

def preprocess_defects(defects):
    """ Preprocess defects into a more efficient data structure for counting. """
    defect_counts = {'a': [0] * dough_length, 'b': [0] * dough_length, 'c': [0] * dough_length}
    for pos, cls in defects:
        if 0 <= pos < dough_length:
            defect_counts[cls][int(pos)] += 1
    return defect_counts



In [82]:
import random

def generate_initial_solution_optimized(defect_counts):
    """ Generate a random initial solution using the optimized defect data structure. """
    solution = []
    position = 0
    while position < dough_length:
        biscuit_type = random.choice(list(biscuits.keys()))
        if is_valid_placement_optimized(position, biscuit_type, defect_counts):
            solution.append((position, biscuit_type))
            position += biscuits[biscuit_type]['length']
        else:
            position += 1
    return solution

def hill_climbing_optimized(defect_counts):
    """ Perform an optimized hill climbing algorithm using the efficient defect data structure. """
    current_solution = generate_initial_solution_optimized(defect_counts)
    current_value = calculate_solution_value(current_solution)
    iterations = 0
    while iterations < 100:  # Limiting iterations for efficiency
        neighbors = []
        for i in range(len(current_solution)):
            position, current_type = current_solution[i]
            for biscuit_type in biscuits:
                if biscuit_type != current_type:
                    new_solution = current_solution[:i] + [(position, biscuit_type)] + current_solution[i+1:]
                    if all(is_valid_placement_optimized(pos, bt, defect_counts) for pos, bt in new_solution):
                        neighbors.append(new_solution)

        best_neighbor = max(neighbors, key=calculate_solution_value, default=current_solution)
        best_neighbor_value = calculate_solution_value(best_neighbor)
        if best_neighbor_value <= current_value:
            break
        current_solution, current_value = best_neighbor, best_neighbor_value
        iterations += 1
    return current_solution

# Execute the optimized hill climbing algorithm with the new defect data structure
optimal_solution_optimized = hill_climbing_optimized(defect_counts)
optimal_solution_optimized, calculate_solution_value(optimal_solution_optimized)


NameError: name 'BISCUITS' is not defined

In [95]:
from collections import namedtuple

Biscuit = namedtuple("Biscuit", ["id", "length", "defects", "value"])

def update_solution(current_solution, position, biscuit):
    total_value = biscuit.value + current_solution[position - biscuit.length].total_value
    path = current_solution[position - biscuit.length].path + [(position - biscuit.length, biscuit.id)]
    return namedtuple("Solution", ["total_value", "path"])(total_value, path)

def hill_climb(biscuits, defect_counts, dough_length, max_iterations=1000):
    Solution = namedtuple("Solution", ["total_value", "path"])

    current_solution = [Solution(0, []) for _ in range(dough_length + 1)]
    best_solution = current_solution

    for _ in range(max_iterations):
        for position in range(1, dough_length + 1):
            for biscuit in biscuits.values():
                length = biscuit.length
                if length <= position:
                    defect_counts_for_biscuit = {
                        k: defect_counts[j][k] for j in range(position - length, position) if j in defect_counts for k in defect_counts[j]
                    }
                    if all(count <= biscuit.defects[k] for k, count in defect_counts_for_biscuit.items()):
                        new_solution = update_solution(current_solution, position, biscuit)
                        current_solution[position] = max(new_solution, current_solution[position - 1], key=lambda s: s.total_value)

        if current_solution[-1].total_value > best_solution[-1].total_value:
            best_solution = current_solution

    return best_solution

# Example usage
biscuits_data = {
    0: {"length": 4, "value": 6, "defects": {"a": 4, "b": 2, "c": 3}, "id": 0},
    1: {"length": 8, "value": 12, "defects": {"a": 5, "b": 4, "c": 4}, "id": 1},
    2: {"length": 2, "value": 1, "defects": {"a": 1, "b": 2, "c": 1}, "id": 2},
    3: {"length": 5, "value": 8, "defects": {"a": 2, "b": 3, "c": 2}, "id": 3},
}

# Convert the dictionary to namedtuples
biscuits = {key: Biscuit(**value) for key, value in biscuits_data.items()}

best_solution = hill_climb(biscuits, defect_counts, dough_length)
print("Best Total Value:", best_solution[-1].total_value)
print("Best Solution:", best_solution[-1].path)


KeyboardInterrupt: 